In [95]:
import pandas as pd
import numpy as np
import sqlite3

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession 


# Basic SQL

In [72]:
con = sqlite3.connect('tickets.db')
cur = con.cursor()

In [73]:
violation_count =  pd.read_sql_query("Select ViolationCode,ViolationDescription, Count(*) AS ViolationCount FROM Parking_Violations_2015 GROUP BY ViolationCode",con)

In [74]:
violation_count.sort_values(by='ViolationCount',ascending=False).head(20)

,ViolationCode,ViolationDescription,ViolationCount
21,21,21-No Parking (street clean),1630912
38,38,38-Failure to Display Muni Rec,1418627
14,14,14-No Standing,988469
36,36,PHTO SCHOOL ZN SPEED VIOLATION,839197
37,37,37-Expired Muni Meter,795918
7,7,None,719753
20,20,20A-No Parking (Non-COM),662904
46,46,46A-Double Parking (Non-COM),595709
71,71,71A-Insp Sticker Expired (NYS),576940
40,40,40-Fire Hydrant,534447


Looks like we don't need to download all the rows of the data - some of the ViolationCodes aren't related to pick-ups - but descriptions aren't clean so downloading violation codes for NYC OpenData

In [75]:
violation_codes =pd.read_json("https://data.cityofnewyork.us/resource/ncbg-6agr.json")
violation_codes

,code,definition,manhattan_96th_st_below,all_other_areas
0,1,FAILURE TO DISPLAY BUS PERMIT,515,515
1,2,NO OPERATOR NAM/ADD/PH DISPLAY,515,515
2,3,UNAUTHORIZED PASSENGER PICK-UP,515,515
3,4,BUS PARKING IN LOWER MANHATTAN,115,115
4,5,BUS LANE VIOLATION,50,50
...,...,...,...,...
92,93,REMOVE/REPLACE FLAT TIRE,65,65
93,96,RAILROAD CROSSING,95,95
94,97,VACANT LOT,65,45
95,98,OBSTRUCTING DRIVEWAY,95,95


In [76]:
# get map definitions from ViolationCode

violation_count['definition']=""
#violation_count.set_index('ViolationCode',inplace=True)
violation_codes.set_index('code',inplace=True)
violation_count.update(violation_codes)
#violation_count.reset_index(drop=True,inplace=True)
violation_codes.reset_index(drop=True,inplace=True)

In [90]:
violation_count['%_of_total']=violation_count['ViolationCount']/violation_count['ViolationCount'].sum()
violation_count.sort_values(by='ViolationCount',ascending=False).head(30)

,ViolationCode,ViolationDescription,ViolationCount,definition,%_of_total
21,21,21-No Parking (street clean),1630912,NO PARKING-STREET CLEANING,0.138105
38,38,38-Failure to Display Muni Rec,1418627,FAIL TO DSPLY MUNI METER RECPT,0.120129
14,14,14-No Standing,988469,NO STANDING-DAY/TIME LIMITS,0.083703
36,36,PHTO SCHOOL ZN SPEED VIOLATION,839197,PHTO SCHOOL ZN SPEED VIOLATION,0.071063
37,37,37-Expired Muni Meter,795918,EXPIRED MUNI METER,0.067398
7,7,None,719753,FAILURE TO STOP AT RED LIGHT,0.060948
20,20,20A-No Parking (Non-COM),662904,NO PARKING-DAY/TIME LIMITS,0.056134
46,46,46A-Double Parking (Non-COM),595709,DOUBLE PARKING,0.050444
71,71,71A-Insp Sticker Expired (NYS),576940,INSP. STICKER-EXPIRED/MISSING,0.048855
40,40,40-Fire Hydrant,534447,FIRE HYDRANT,0.045257


# Pyspark

In [4]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('Milestone1') \
    .getOrCreate()
sc = spark.sparkContext

NameError: name 'SparkSession' is not defined